# Methods for use throughout the process

## Contents:<a class="anchor" id="contents"></a>
* [Model building and compiling](#modelbuilding)
* [Fitting Models to Generators](#fitting)
* [Plotting Results](#plotting)
* [Miscellaneous](#misc)
* [Model Ensembling](#ensembling)

In [3]:
import time
import tensorflow as tf
import keras.backend as K
import numpy as np
import pandas as pd
from keras import models, layers, optimizers, regularizers
from keras.models import Model, load_model
from keras.preprocessing.image import ImageDataGenerator
from keras.preprocessing import image
from keras import Input
import matplotlib.image as mpimg
import matplotlib.pyplot as plt
import keras

from keras_gcnn.layers import GConv2D, GBatchNorm
from keras_gcnn.layers.pooling import GroupPool

from keras.utils import np_utils
from keras.layers import concatenate
from keras.layers import Activation
from keras.layers import Cropping2D # Need this for concatenate to work with valid pooling
from keras.regularizers import l2 # Weight decay from the DenseNet paper

## Model building and compiling <a class="anchor" id="modelbuilding"></a>
----------------------------------

##### This method was adapted from my own course work for the Artificial Intelligence Module

In [1]:
def build_and_compile_model(input_layer, hidden_layer_units=None, output = "sigmoid",
                            img_size=96,labels = 1, opt="Adam",  learn = 1e-3, convpad = "same",
                            poolpad="valid", metrics = ["accuracy"],weight_reg=1e-4):
    """Takes an input layer unit size and several other optional tuning parameters and returns a compiled Keras model.
    
    This method automates part of the Keras model building and compiling process, allowing a user to simply
    give their hyper-parameters and model characteristics as arguments in one function. It has set defaults 
    relative to this specific dataset currently.
    
    It makes use of the functional Keras API rather than the sequential Model class.
    
    """
    input_tensor = Input(shape=(img_size,img_size,3))
    x = layers.Conv2D(input_layer, (3,3), activation="relu", padding=convpad) (input_tensor)
    
    if hidden_layer_units is not None:
        for u in hidden_layer_units:
            if isinstance(u, str):
                if "conv" in u:
                    if "reg" in u:
                        x = layers.Conv2D(int(u[7:]), (3,3), activation = "relu",
                                          padding=convpad, kernel_regularizer=regularizers.l2(weight_reg)) (x)
                    else:
                        x = layers.Conv2D(int(u[4:]), (3,3), activation = "relu", padding=convpad) (x)
                elif "dropout" in u:
                    x = layers.Dropout(float(u[7:])) (x)
                elif "maxpool" in u:
                    x = layers.MaxPooling2D((2,2), padding=poolpad) (x)
                elif "globalavg" in u:
                    x = layers.GlobalAveragePooling2D() (x)
                elif "globalmax" in u:
                    x = layers.GlobalMaxPooling2D() (x)
                elif "flatten" in u:
                    x = layers.Flatten() (x)
                elif "norm" in u:
                    x = layers.BatchNormalization() (x)
                else:
                    if u != "":
                        print("Invalid argument for hidden_layer_units: {}".format(u))
            else:
                x = layers.Dense(u, activation = "relu") (x)
    
    output_tensor = layers.Dense(labels, activation = output) (x)
    
    model = Model(input_tensor, output_tensor)
    if output == "sigmoid":
        l = "binary_crossentropy"
    else:
        l = "categorical_crossentropy"
    if opt == "RMS":
        model.compile(optimizer = optimizers.RMSprop(lr = learn),
            loss = l,
            metrics = metrics)
    elif opt == "Adam":
        model.compile(optimizers.Adam(lr = learn),
            loss = l,
            metrics = metrics)                
    else:
        print("Invalid argument for 'opt'")
        return
    return model

#### GCNN Model

In [3]:
def build_and_compile_model_GCNN(gconv_type = "C4", input_layer = 32, hidden_layer_units=None,
                                 output = "sigmoid", labels = 1, img_size=96, opt="Adam",  learn = 1e-3,
                                 poolpad="valid", metrics = ["accuracy"], weight_reg=1e-4):
    """Takes an input layer unit size and several other optional tuning parameters and returns a compiled Keras model.
    
    This is adapted from my standard building and compilation function to work solely with non-standard
    Group Convolutional Neural Networks (G-CNNs)
    
    The layers utilised are extended from the built-in Keras layers to exploit rotational and reflectional symmetries.
    
    gconv_type specifies the specific gconv operation to use.
    "C4" contains the four pure 90 degree rotations
    "D4" contains the eight roto-reflections
    
    
    
    """
    input_tensor = Input(shape=(img_size,img_size,3))
    x = GConv2D(input_layer, 3, "Z2", gconv_type, activation="relu", padding="same") (input_tensor)
    
    if hidden_layer_units is not None:
        for u in hidden_layer_units:
            if isinstance(u, str):
                if "conv" in u:
                    if "reg" in u:
                        x = GConv2D(int(u[7:]), 3, gconv_type, gconv_type, activation = "relu",
                                    padding="same", kernel_regularizer=regularizers.l2(weight_reg)) (x)
                    else:
                        x = GConv2D(int(u[4:]), 3, gconv_type, gconv_type, activation = "relu", padding="same") (x)
                elif "dropout" in u:
                    x = layers.Dropout(float(u[7:])) (x)
                elif "maxpool" in u:
                    x = layers.MaxPooling2D((2,2), padding=poolpad) (x)
                elif "grouppool" in u:
                    x = GroupPool(h_input=gconv_type)(x)
                elif "globalavg" in u:
                    x = layers.GlobalAveragePooling2D() (x)
                elif "globalmax" in u:
                    x = layers.GlobalMaxPooling2D() (x)
                elif "flatten" in u:
                    x = layers.Flatten() (x)
                elif "norm" in u:
                    x = GBatchNorm(gconv_type)(x)
                else:
                    if u != "":
                        print("Invalid argument for hidden_layer_units: {}".format(u))
            else:
                x = layers.Dense(u, activation = "relu") (x)
    
    output_tensor = layers.Dense(labels, activation = output) (x)
    
    model = Model(input_tensor, output_tensor)
    if output == "sigmoid":
        l = "binary_crossentropy"
    else:
        l = "categorical_crossentropy"
    if opt == "RMS":
        model.compile(optimizer = optimizers.RMSprop(lr = learn),
            loss = l,
            metrics = metrics)
    elif opt == "Adam":
        model.compile(optimizers.Adam(lr = learn),
            loss = l,
            metrics = metrics)                
    else:
        print("Invalid argument for 'opt'")
        return
    return model

#### DenseNet model builder functions

In [1]:
def build_and_compile_dense_model(filters,growable_filters=None,growth_rate=3,dense_blocks=5,is_gconv=True,gconv_type="C4",conv_layers=1,
                                  padding="valid",pooling="avg",dropout=0,norm=True,labels=1,img_size=96,
                                  output="sigmoid",opt="adam",learn=1e-3,metrics=["accuracy"],weight_decay=1e-4,
                                 bc_model=False):
    
    """ Takes as input :the initial filters, growth rate, dense block structure, g_conv type, and several others. 
    
    This method provides the ability to automatically construct DenseNet models, either with standard convolutions
    or with group-equivariant layers. It also has the option to include bottleneck layers and compression, as described 
    in the DenseNet paper.
    
    
    """
    # Firstly defining the dense and transition blocks as nested functions.
    
    def dense_block(input_layer,filters,growth_rate,is_gconv=True,gconv_type="C4",conv_layers=1,
                padding="valid",dropout=0,norm=True,weight_decay=1e-4,bc_model=False):
    
        """ 
        Used by its parent function to create the individual dense blocks for the overall network. 
        

        """        
        x = input_layer
        for i in range(conv_layers):

            #if BC model version of DenseNet - add a "bottleneck" conv layer
            if bc_model:
                if norm:
                    if is_gconv:
                        conv = GBatchNorm("C4") (x)
                    else:
                        conv = layers.BatchNormalization() (x)
                    conv = Activation("relu") (conv)
                else:
                    conv = Activation("relu") (x)
                if is_gconv:
                    conv = GConv2D(growth_rate*4, 1, gconv_type, gconv_type, padding=padding, kernel_regularizer=l2(weight_decay))(conv)
                else:
                    conv = layers.Conv2D(growth_rate*4, (1,1), padding=padding, kernel_regularizer=l2(weight_decay)) (conv)
                if norm:
                    if is_gconv:
                        conv = GBatchNorm("C4") (conv)
                    else:
                        conv = layers.BatchNormalization() (conv)
            else:
                if norm:
                    if is_gconv:
                        conv = GBatchNorm("C4") (x)
                    else:
                        conv = layers.BatchNormalization() (x)
                    conv = Activation("relu") (conv)
                else:
                    conv = Activation("relu") (x)

            if is_gconv:
                conv = GConv2D(growth_rate, 3, gconv_type, gconv_type, padding=padding, kernel_regularizer=l2(weight_decay))(conv)
            else:
                conv = layers.Conv2D(growth_rate, (3,3), padding=padding, kernel_regularizer=l2(weight_decay)) (conv)
            if dropout > 0:
                conv = layers.Dropout(dropout) (conv)

            x = concatenate([crop(x,conv),conv])

            nonlocal growable_filters
            growable_filters += growth_rate

        return x
    
    def transition_block(input_layer,filters,is_gconv=True,gconv_type="C4",
                     padding="valid",pooling="avg",norm=True,weight_decay=1e-4,bc_model=False):
    
        """ 
        Used by its parent function to create individual transition blocks for use between dense blocks and
        reduce the dimensionality of the network.

        """


        x = input_layer

        #Bottleneck and compression model of DenseNet.
        nonlocal growable_filters
        if bc_model:
            filters = int(round(growable_filters*0.5))
        else:
            filters = growable_filters

        if norm:
            if is_gconv:
                x = GBatchNorm("C4") (x)
            else:
                x = layers.BatchNormalization() (x)

        x = Activation("relu") (x)

        if is_gconv:
            x = GConv2D(filters, 1, gconv_type, gconv_type, padding=padding, kernel_regularizer=l2(weight_decay)) (x)
        else:
            x = layers.Conv2D(filters, (1,1), padding=padding, kernel_regularizer=l2(weight_decay)) (x)

        if pooling == "avg":
            x = layers.AveragePooling2D((2,2)) (x)
        elif pooling == "max":
            x = layers.MaxPooling2D((2,2)) (x)
        return x
    
    # Start of model building process
    
    if growable_filters==None:
        growable_filters = filters
    
    input_tensor = Input(shape=(img_size,img_size,3))
    if is_gconv:
        x = GConv2D(filters, 3, "Z2", gconv_type, activation="relu", padding=padding,  kernel_regularizer=l2(weight_decay)) (input_tensor)
    else:
        x = layers.Conv2D(filters, (3,3), activation="relu", padding=padding,  kernel_regularizer=l2(weight_decay)) (input_tensor)
    
    for i in range(dense_blocks-1): # -1 because the final block doesn't have a transition.
        x = dense_block(x,filters,growth_rate,is_gconv,gconv_type,conv_layers,padding,dropout,norm,weight_decay,bc_model)
        x = transition_block(x,filters,is_gconv,gconv_type,padding,pooling,norm,weight_decay,bc_model)
 
    # Final dense block doesn't have a transition block after it.
    x = dense_block(x,filters,growth_rate,is_gconv,gconv_type,conv_layers,padding,dropout,norm,weight_decay,bc_model)
    
    if norm:
        if is_gconv:
            x = GBatchNorm(gconv_type) (x)
        else:
            x = layers.BatchNormalization() (x)
        
    x = Activation("relu") (x)
    
    # If it's a Gconv model, we need to group pool before the global pooling to maintain equivariance.
    if is_gconv:
        x = GroupPool(h_input=gconv_type)(x)
    
    #Global pooling to replace the flattening layer and any dense layers prior to classification.
    x = layers.GlobalAveragePooling2D() (x)
    
    output_tensor = layers.Dense(labels, activation = output) (x)
    model = Model(input_tensor,output_tensor)
    
    #Compiling the model
    if labels==1:
        lossfunc = "binary_crossentropy"
    else:
        lossfunc = "categorical_crossentropy"
    
    if opt == "RMS":
        model.compile(optimizer = optimizers.RMSprop(lr = learn),
            loss = lossfunc,
            metrics = metrics)
    elif opt == "Adam":
        model.compile(optimizers.Adam(lr = learn),
            loss = lossfunc,
            metrics = metrics)
    elif opt == "DenseSGD":
        # Using the specific settings described in the DenseNet paper.
        model.compile(optimizer = optimizers.SGD(lr = 0.1,momentum=0.9,nesterov=True),
                     loss = lossfunc,
                     metrics = metrics)
    
    
    return model
    

#### Cropping convolutional layers
##### Required to concatenate two varying sized conv layers. Needed for the models in the rotational paper as they use "valid" padding.
##### This function was adapted from https://github.com/basveeling/keras-gcnn/tree/master/keras_gcnn - access date: 26/02/2019

In [ ]:
## This function was adapted from https://github.com/basveeling/keras-gcnn/tree/master/keras_gcnn
def crop(inp,to_fit):
    
    input_layer = inp
    skip_size = K.int_shape(input_layer)[1]
    out_size = K.int_shape(to_fit)[1]
    if skip_size > out_size:
        size_diff = (skip_size - out_size) // 2
        size_diff_odd = ((skip_size - out_size) // 2) + ((skip_size - out_size) % 2)
        input_layer = Cropping2D(((size_diff, size_diff_odd),) * 2)(input_layer)
    return input_layer

## Model fitting to generator <a class="anchor" id="fitting"></a>
----------------------------------

In [4]:
def fit_model_to_generator(model,generator, x, y, vx, vy, callbacks=None, steps=None, batch_size=32, epochs = 25, validation_steps = 50, verbose = 1):
    if steps==None:
        steps=len(x)//batch_size
    return model.fit_generator(
        generator=generator.flow(x,y,batch_size),
        steps_per_epoch = steps,
        epochs = epochs,
        validation_data = (vx,vy),
        callbacks = callbacks,
        verbose = verbose)

In [1]:
def fit_model_to_directory_generator(model, generator, direct, val_gen, tar_size=(96,96), class_mode="binary", batch_size=32, callbacks=None, steps=100, val_steps=None, epochs = 25, verbose = 1):
    if val_steps==None:
        val_steps=len(val_gen)-1
    return model.fit_generator(
        generator.flow_from_directory(direct,
                                      target_size=tar_size,
                                      batch_size=batch_size,
                                      class_mode=class_mode),        
        steps_per_epoch = steps,
        epochs = epochs,
        validation_data=val_gen,
        validation_steps = val_steps,
        callbacks = callbacks,
        verbose = verbose)

# Plotting Results <a class="anchor" id="plotting"></a>
----------------------------------

##### Adapted from my own coursework for Artificial Intelligence

In [128]:
def plot_results(historyID,metric="acc",start=0):
    """Plotting the metric results of Keras history objects using matplotlib.

    This method takes a Keras history object and plots either the loss (blue circles) and validation loss (blue line)
    or the accuracy (blue circles) and validation accuracy (blue line). It can start from non-zero on the x axis should
    the user want to localise their results.
    
    In addition it plots the smoothed version of the validation set alongside for a clearer view of the curve.

    """
    plt.clf()

    history_dict = historyID.history

    fig, ax = plt.subplots(1,2, figsize=(15,5))
    x = list(range(start+1,len(history_dict["val_loss"])+start+1))
    
    if metric == "acc":
        ymax = max(history_dict["val_acc"])
        xmax = history_dict["val_acc"].index(ymax)

        ax[0].plot(x,history_dict["val_acc"][start:], "b", label ="Validation acc")
        ax[0].plot(x,history_dict["acc"][start:],"bo", label = "Training acc")
        ax[0].set_xlabel("Epochs")
        ax[0].set_ylabel("Accuracy")
        ax[0].set_title("Training and validation accuracy")
        ax[0].legend()

        anno_text = "Highest val accuracy: x= {}, y= {}".format(xmax, ymax)
        
        plt.subplot(122)
        plot_smooth(historyID,metric="val_acc")
        
        
    elif metric == "loss":
        ymin = min(history_dict["val_loss"])
        xmin = history_dict["val_loss"].index(ymin)
    
        ax[0].plot(x,history_dict["val_loss"][start:], "b", label ="Validation loss")
        ax[0].plot(x,history_dict["loss"][start:],"bo", label = "Training loss")
        ax[0].set_xlabel("Epochs")
        ax[0].set_ylabel("Loss")
        ax[0].set_title("Training and validation loss")
        ax[0].legend()
        
        anno_text = "Lowest val loss: x= {}, y= {}".format(xmin, ymin)
        
        plt.subplot(122)
        plot_smooth(historyID,metric="val_loss")      
        
    else:
        print("Please enter either 'acc', or 'loss' as the second parameter.")
        return
        

    plt.show()
    print(anno_text)
    
def plot_graphs(historyID,start=0):
    plot_results(historyID,"acc",start)
    plot_results(historyID,"loss",start)

In [7]:
def plot_smooth(historyID, factor = 0.9, metric = "val_acc"):  # ACC as primary metric currently
    smoothed_points = []
    for point in historyID.history[metric]:
        if smoothed_points:
            previous = smoothed_points[-1]
            smoothed_points.append(previous * factor + point * (1 - factor))
        else:
            smoothed_points.append(point)
    plt.plot(range(1, len(smoothed_points) +1), smoothed_points)
    plt.title("smooth " + metric)
    plt.xlabel("Epochs")
    plt.ylabel(metric)
    return plt

def plot_smooth_graphs(historyID, factor = 0.9):
    plot_smooth(historyID, factor, metric="val_acc")
    plot_smooth(historyID, factor, metric="val_loss")

#### AUC/ROC calculation and plotting

#### Utilising the SKlearn ROC_curve metric and basing the plot off of https://scikit-learn.org/stable/auto_examples/model_selection/plot_roc.html

In [1]:
from sklearn.metrics import roc_curve

def evaluate_auc(model,generator,batches=None):
    if batches==None:
        batches = len(generator)-1
    predictions = []
    test_labels = []
    count = 0
    for img_batch,label_batch in generator:
        test_labels.extend(label_batch)
        predictions.extend(model.predict(img_batch))
        count+=1
        if count >= batches:
            break
    plot_auc(test_labels,predictions)
        
# https://scikit-learn.org/stable/auto_examples/model_selection/plot_roc.html - based off the example shown.
def plot_auc(y,y_pred):
    plt.clf()
             
    dashed_green_line = "g--"
    fig, ax = plt.subplots(1,1, figsize=(7,5))
    
    false_positive_rate, true_positive_rate = roc_curve(y,y_pred)[:2]
    label = "AUC: %.3f" % area_under_ROC_curve(false_positive_rate,true_positive_rate)
    ax.plot(false_positive_rate,true_positive_rate, color = "red",label=label)
    ax.plot([0,1],[0,1],dashed_green_line)
    ax.fill_between(ax.lines[0].get_xydata()[:,0],ax.lines[0].get_xydata()[:,1],color = "cyan",alpha=0.2)
    plt.xlim([0.0,1.0])
    plt.ylim([0.0,1.05])
    ax.legend(loc=4)
    ax.set_ylabel("True positive rate")
    ax.set_xlabel("False positive rate")
    ax.set_title("Receiver Operating Characteristic Curve")
    
    plt.show()
    
def area_under_ROC_curve(false_positive_rate,true_positive_rate):
    # Using the trapezoid rule for calculating area
    # based off https://scikit-learn.org/stable/modules/generated/sklearn.metrics.auc.html
    # and discussed in http://www.med.mcgill.ca/epidemiology/hanley/software/Hanley_McNeil_Radiology_82.pdf
    return np.trapz(true_positive_rate,false_positive_rate)


    

## Miscellaneous <a class="anchor" id="misc"></a>
----------------------------------

In [42]:
# A function to check whether the model is rotationally equivariant.
# It generates predictions on a sample from the training set and a rotated version of that sample.
# If these return the same result, the model is equivariant.

# Concatenates the image and its rotated variants into one numpy array.
# Then passes the array through the model for predictions
# and checks whether all predictions are virtually identical.

def equi_check(model,X):
    # Concatenating the image and its rotated variant into 1 numpy array.
    imgs = np.array([X[0], np.rot90(X[0]),np.rot90(np.rot90(X[0])),np.rot90(np.rot90(np.rot90(X[0])))])
    # Passing the array through the model for predictions
    predictions = model.predict(imgs)
    # Checking whether all predictions are virtually identical.
    return np.allclose(predictions[0], predictions[1]) 

#### Model memory required 
#####  Adapted from https://stackoverflow.com/questions/43137288/how-to-determine-needed-memory-of-keras-model - access date: 27/02/2019

In [2]:
#  Adapted from https://stackoverflow.com/questions/43137288/how-to-determine-needed-memory-of-keras-model 
def memory_required(model,b_size):
    shapes_count = int(np.sum([np.prod(np.array([s if isinstance(s, int) else 1 for s in l.output_shape])) for l in model.layers]))
    memory = shapes_count * 4
    gbytes = np.round(memory * b_size / (1024.0 ** 3), 3)
    return (gbytes)

#### Counting the convolutional layers

In [ ]:
def count_conv_layers(model):
    sum=0
    for l in model.layers:
        if "conv" in l.get_config()["name"]:
            sum+=1
    return sum

#### Saving and loading the history object

In [1]:
import pickle
def save_history(history,history_string):
    pickle_out = open("C:/GitRepos/FINAL PROJECT DATA/Histopathologic Cancer Detection/histories/{}.pickle".format(history_string),"wb")
    pickle.dump(history,pickle_out)
    pickle_out.close()

def load_history(history_string):
    pickle_in = open("C:/GitRepos/FINAL PROJECT DATA/Histopathologic Cancer Detection/histories/{}.pickle".format(history_string),"rb")
    return pickle.load(pickle_in)

## Model ensembling <a class="anchor" id="ensembling"></a>
----------------------------------

In [1]:
#Evaluates the ensemble and the original models for comparison.
def model_ensemble_evaluation(models,test_gen,steps=None,weighted=False):
    """ Takes a list of models, a data generator and optional weighting as input.
     Ensembles the models and performed rank-averaging (if chosen) weighting on them based off a small
     validation set. It then makes predictions on the test set and pools them together.    
    
    """
    
    if steps==None:
        steps = len(test_gen)-1
    
    test_sample = []
    test_labels_sample = []
    val_sample = []
    val_labels_sample = []
    
    for i in range(steps):
        next_batch = test_gen.next()
        if weighted and i<steps*0.2:
            val_sample.extend(next_batch[0])
            val_labels_sample.extend(next_batch[1])
        else:
            test_sample.extend(next_batch[0])
            test_labels_sample.extend(next_batch[1])
   
    #converting to numpy array
    test_sample = np.array(test_sample)
    test_labels_sample=np.array(test_labels_sample)
    val_sample = np.array(val_sample)
    val_labels_sample = np.array(val_labels_sample)
    
    pred_list = []
    for m in models:
        pred_list.append(m.predict(test_sample))
    
    # If not utilising the weighted predictions, simply average them out.
    if not weighted:
        ensembled_pred = np.sum(pred_list,axis=0)/len(models)
    else:
        val_scores = []
        for m in models:
            # For each model, evaluate a validation score.
            val_scores.append(m.evaluate(val_sample,val_labels_sample)[1])
        val_sum = np.sum(val_scores)
        print("val scores: ", val_scores)
        weights = []
        for i in val_scores:
        # Normalise the weights based on the validation scores.
            weights.append(i/val_sum)
        print("weights",weights)
        ensembled_pred = np.zeros((len(test_labels_sample),1))
        for count, p in enumerate(pred_list):
            # Sum up the weighted predictions.
            ensembled_pred += p*weights[count]     
    
    
    pred_list.append(ensembled_pred)
    return acc_comparison(pred_list,test_labels_sample)

from sklearn.metrics import roc_curve
def acc_comparison(predictions_list,labels):
    accuracies=[]
    AUCs=[]
    for preds in predictions_list:
        false_positive_rate, true_positive_rate = roc_curve(labels,preds)[:2]
        AUCs.append(area_under_ROC_curve(false_positive_rate,true_positive_rate))
        correct = 0
        for i in range(len(labels)):
            if int(round(preds[i][0])) == labels[i]:
                correct+=1
        acc = correct/len(labels)
        accuracies.append(acc)
    return accuracies,AUCs